In [1]:
import pandas as pd
import numpy as np 
import requests 
import shutil 
import threading

In [5]:
import os
path = os.path.abspath(".")
os.path.split(path)
len("Open-Food-Fact")

14

In [8]:
l = os.path.abspath(".")
l[:l.find("Open-Food-Fact") + len("Open-Food-Fact")]

'c:\\Users\\Utilisateur\\Documents\\HETIC\\Hub_hetic\\'

In [2]:
# chemin base de données avec url image
df = pd.read_csv("../data/Data_a_scrap/data_url_image_train_110k_v2.csv")
df.head()

,Unnamed: 0,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,countries,countries_tags,countries_en,image_url
0,0,000000000114,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1580066482,2020-01-26T19:21:22Z,1609945229,2021-01-06T15:00:29Z,Chocolate n 3,en:fr,en:france,France,https://static.openfoodfacts.org/images/produc...
1,1,0000000005470,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1511778860,2017-11-27T10:34:20Z,1550180518,2019-02-14T21:41:58Z,BAguette bressan,France,en:france,France,https://static.openfoodfacts.org/images/produc...
2,2,000000012753,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1562659733,2019-07-09T08:08:53Z,1599581974,2020-09-08T16:19:34Z,Barre chocolat au lait praliné,en:fr,en:france,France,https://static.openfoodfacts.org/images/produc...
3,3,0000000274739,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1513030610,2017-12-11T22:16:50Z,1513097739,2017-12-12T16:55:39Z,Raviolini au Fromage de chèvre et Pesto,France,en:france,France,https://static.openfoodfacts.org/images/produc...
4,4,0000000491228,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1543690964,2018-12-01T19:02:44Z,1543692731,2018-12-01T19:32:11Z,Entremets Crème Brulée,France,en:france,France,https://static.openfoodfacts.org/images/produc...


In [3]:
class myThread (threading.Thread):
    def __init__(self, threadID, name, q):
        threading.Thread.__init__(self)
        self.threadID = threadID
        self.name = name
        self.q = q
    
    def run(self):
        print ("Starting " + self.name)
        process_data(self.name, self.q)
        print ("Exiting " + self.name)

In [7]:
# scrapper les images train
import queue
import threading
import time

list_url = df['image_url'].to_numpy()
list_code = df['code'].to_numpy()
print(len(list_url))
exitFlag = 0

# fonction pour télécharger une image
def process_data(threadName, q):
    while not exitFlag:
        queueLock.acquire()
        if not workQueue.empty():
            loic, nom = q.get()
        queueLock.release()
        r = requests.get(loic, stream=True)
        if r.status_code != 200:
            with open("image_non_telecharger.txt", "w") as fichier:
                fichier.write(f"image not télécharger code : {nom}\n")
        filename = "../data/image/" + str(nom) + "." +  loic.split('.')[-1]
        with open(filename, 'wb') as f:
            shutil.copyfileobj(r.raw, f)

threadList = ["Thread-1", "Thread-2", "Thread-3", "Thread-4", "Thread-5", "Thread-6", "Thread-7", "Thread-8"]
queueLock = threading.Lock()
queueLock.acquire()
workQueue = queue.Queue(len(list_code))
threads = []
threadID = 1

print("Create new threads")
# Create new threads
for tName in threadList:
    thread = myThread(threadID, tName, workQueue)
    thread.start()
    threads.append(thread)
    threadID += 1

print("Fill the queue")
# Fill the queue
#queueLock.acquire()
for coup in zip(list_url[37926:], list_code[37926:]):
    workQueue.put(coup)
queueLock.release()

print("Wait the queue")
# Wait for queue to empty
while not workQueue.empty():
    pass

# Notify threads it's time to exit
exitFlag = 1

# Wait for all threads to complete
for t in threads:
    t.join()
print ("Exiting Main Thread")

108980
Create new threads
Starting Thread-1
Starting Thread-2
Starting Thread-3
Starting Thread-4
Starting Thread-5
Starting Thread-6
Starting Thread-7
Starting Thread-8
Fill the queue
Wait the queue
Exiting Thread-8
Exiting Thread-1
Exiting Thread-3
Exiting Thread-4
Exiting Thread-7
Exiting Thread-2
Exiting Thread-6
Exiting Thread-5
Exiting Main Thread


In [8]:
# scrapper les images test
# chemin base de données avec url image
df = pd.read_csv("../data/Data_a_scrap/data_url_image_test_110k_v2.csv")
# scrapper les images train
import queue
import threading
import time

list_url = df['image_url'].to_numpy()
list_code = df['code'].to_numpy()
exitFlag = 0

# fonction pour télécharger une image
def process_data(threadName, q):
    while not exitFlag:
        queueLock.acquire()
        if not workQueue.empty():
            loic, nom = q.get()
        queueLock.release()
        r = requests.get(loic, stream=True)
        if r.status_code != 200:
            with open("image_non_telecharger.txt", "w") as fichier:
                fichier.write(f"image not télécharger code : {nom}\n")
        filename = "../data/CNN/TEST/" + str(nom) + "." +  loic.split('.')[-1]
        with open(filename, 'wb') as f:
            shutil.copyfileobj(r.raw, f)

threadList = ["Thread-1", "Thread-2", "Thread-3", "Thread-4", "Thread-5", "Thread-6", "Thread-7", "Thread-8"]
queueLock = threading.Lock()
queueLock.acquire()
workQueue = queue.Queue(len(list_code))
threads = []
threadID = 1

print("Create new threads")
# Create new threads
for tName in threadList:
    thread = myThread(threadID, tName, workQueue)
    thread.start()
    threads.append(thread)
    threadID += 1

print("Fill the queue")
# Fill the queue
#queueLock.acquire()
for coup in zip(list_url, list_code):
    workQueue.put(coup)
queueLock.release()

print("Wait the queue")
# Wait for queue to empty
while not workQueue.empty():
    pass

# Notify threads it's time to exit
exitFlag = 1

# Wait for all threads to complete
for t in threads:
    t.join()
print ("Exiting Main Thread")


Create new threads
Starting Thread-1
Starting Thread-2
Starting Thread-3Starting Thread-4
Starting Thread-5

Starting Thread-6
Starting Thread-7
Starting Thread-8Fill the queue

Wait the queue
Exiting Thread-8
Exiting Thread-3
Exiting Thread-2
Exiting Thread-1Exiting Thread-6

Exiting Thread-5Exiting Thread-4

Exiting Thread-7
Exiting Main Thread


fini
salut
